In [1]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

import parallelism
parallelism.set_max_num_tbb_threads(1)

In [8]:
# Load the centerline from file...
fileA = "../data/L400-r0.2-UpTo9Crossings/4_1/0001.obj"
fileB = '../data/L400-r0.2-UpTo9Crossings/4_1/0033.obj'
#fileA = '../data/NoCollision/reduced0001.obj'
#fileB = '../data/NoCollision/reduced0033.obj'
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
B = read_nodes_from_file(fileA)[::4]
A = read_nodes_from_file(fileB)[::4]
pr = define_periodic_rod(A, material)
rod_list = elastic_knots.PeriodicRodList([pr])
nPoints = len(A)
print(B.shape)
problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.hasCollisions = True
problemOptions.contactStiffness = 10000
problemOptions.dHat = 2*rod_radius

contactProblem = elastic_knots.ContactProblem(rod_list,problemOptions)

(100, 3)


In [9]:
view = Viewer(rod_list, width=1024, height=800)

view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [10]:
def moveTo(p, target):
    k = 2
    F = k*(target - p)
    return np.array(F)

def getSpringForce(R, R_next, k=2.0):
    d = R_next - R
    F = k*d 
    return F
        
def sliding(a,b, target):
    o = np.cross(a,b)
    if np.dot(o,target)<0:
        o = -o
    return o/ np.linalg.norm(o)

"""
laplace = points[i - 1] - 2 * points[i] + points[i + 1]

if np.linalg.norm(contactForce[i]) > 80:
            print("siliding")
            F+= sliding(contactForce[i],points[i-1]-points[i],B[i])
            
            
            else:
                break
        for j in range(nPoints-1,0,-1):
            if np.linalg.norm( points[j] - target[j]) < 4.0:
                randVal[j] = target[j]
                print(f"holding {j}")
            else:
                break
                
                
                
                if j < 1500 and j > 1300:
            F = contactForce[i] + 0.01*springForce
            if np.linalg.norm(contactForce[i]) > 0:
                sli = 10* sliding(contactForce[i],points[i-1]-points[i],B[i])
                print(f"siliding {sli}")
                F+= sli
           
"""


'\nlaplace = points[i - 1] - 2 * points[i] + points[i + 1]\n\nif np.linalg.norm(contactForce[i]) > 80:\n            print("siliding")\n            F+= sliding(contactForce[i],points[i-1]-points[i],B[i])\n            \n            \n            else:\n                break\n        for j in range(nPoints-1,0,-1):\n            if np.linalg.norm( points[j] - target[j]) < 4.0:\n                randVal[j] = target[j]\n                print(f"holding {j}")\n            else:\n                break\n                \n                \n                \n                if j < 1500 and j > 1300:\n            F = contactForce[i] + 0.01*springForce\n            if np.linalg.norm(contactForce[i]) > 0:\n                sli = 10* sliding(contactForce[i],points[i-1]-points[i],B[i])\n                print(f"siliding {sli}")\n                F+= sli\n           \n'

In [11]:
j=0
path = []
for it in range(50000):
    DoFs = contactProblem.getDoFs().copy()
    points = DoFs[:nPoints*3].reshape((-1, 3))
    grad = contactProblem.contactForces()
    contactForce = grad[:nPoints*3].reshape((-1, 3))
    
    if j == 0:
        target = B
        
    elif j== 1500:
        minVal = np.min(B)
        maxVal = np.max(B)
        randVal = np.random.rand(*B.shape) * (maxVal-minVal) + minVal
        for i in range(nPoints):
            if np.linalg.norm( points[i] - target[i]) < 4.0:
                randVal[i] = target[i]
                print(f"holding {i}")
            
        target = randVal
    elif j == 2500:
        j = -1
    j += 1
    
    for i in range(0, nPoints):
        
        if i == nPoints-1:
            springForce = getSpringForce(points[i], points[0]) + getSpringForce(points[i], points[i-1])
        else:
            springForce = getSpringForce(points[i], points[i+1]) + getSpringForce(points[i], points[i-1])
                
        F = 0.1*moveTo(points[i], target[i]) +contactForce[i] + springForce
        
        
        
        F = F/ np.linalg.norm(F)    
        points[i] = (points[i] +0.01*F)   
        
    DoFs[:nPoints*3]=points.flatten()
    contactProblem.setDoFs(DoFs.flatten())
    
    if it % 100 == 0 :
        print(f"{it} Dist to Target: {np.linalg.norm(points-target)}")

    if it % 25 == 0:
        path.append(contactProblem.getDoFs())
        
    if(np.linalg.norm(points-B) <100):
        print("Found a solution")
        break
    
    view.update()

0 Dist to Target: 425.01295324406954
100 Dist to Target: 415.5528021396438
200 Dist to Target: 406.1943164242364
300 Dist to Target: 396.8534572143089
400 Dist to Target: 387.6148512782841
500 Dist to Target: 378.49229653959037
600 Dist to Target: 369.4484517152478
700 Dist to Target: 360.5124899427399
800 Dist to Target: 351.6538993992997
900 Dist to Target: 342.87645003487444
1000 Dist to Target: 334.17943966547944
1100 Dist to Target: 325.5653836382325
1200 Dist to Target: 317.0298975922359
1300 Dist to Target: 308.47449764190344
1400 Dist to Target: 300.0301956814658
1500 Dist to Target: 387.72986774413386
1600 Dist to Target: 379.43892548510445
1700 Dist to Target: 372.13428588675765
1800 Dist to Target: 365.39586777983516
1900 Dist to Target: 359.0130438294058
2000 Dist to Target: 352.96642613318915
2100 Dist to Target: 347.2191293534709
2200 Dist to Target: 341.75473285113935
2300 Dist to Target: 336.56857349796616
2400 Dist to Target: 331.70817216737316
2500 Dist to Target: 327

In [12]:
#loaded = np.load('path_good.npz')
#path = [loaded[key] for key in loaded.files][:110]
print(len(path))
contactProblem.setDoFs(path[-1])

356


In [13]:
from helpers import write_obj
'''
#file = '../data/test1.obj'
file = '../data/L400-r0.2-UpTo9Crossings/4_1/0033.obj'
#write_obj(file, rod_list)
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
A = read_nodes_from_file(file)
pr = define_periodic_rod(A, material)
rod_list = elastic_knots.PeriodicRodList([pr])
rod_list.setDoFs(path[-1])
'''
view = Viewer(rod_list, width=1024, height=800)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [14]:

time.sleep(2)
relaxPath = []
def callback(problem, iteration):
    if iteration % 20 == 0:
        relaxPath.append(rod_list.getDoFs())
        view.update()

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = []   # all the degrees of freedom can be modified by the optimizer
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 10000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

report = elastic_knots.compute_equilibrium(
rod_list, problemOptions, optimizerOptions, 
externalForces=np.zeros(rod_list.numDoF()),
softConstraints=[],
callback=callback,
hessianShift=hessianShift
)
view.update()

0	4607.66	574.076	574.076	0.75	1
1	4364.54	417.961	417.961	1	1
2	4304.55	353.81	353.81	1	1
3	4201.85	336.555	336.555	1	1
4	4015.37	320	320	1	1
5	3688.64	297.461	297.461	1	1
6	3538.79	288.1	288.1	1	1
7	3263.66	274.345	274.345	1	1
8	2729.52	262.575	262.575	1	1
9	2494.16	253.254	253.254	1	1
10	2052.21	236.093	236.093	1	1
11	1863.56	226.649	226.649	1	1
12	1486.43	218.549	218.549	1	1
13	1323.67	209.312	209.312	1	1
14	1044.35	181.106	181.106	1	1
15	668.992	133.337	133.337	1	1
16	556.461	120.689	120.689	1	1
17	370.306	97.2688	97.2688	1	1
18	219.078	76.0305	76.0305	1	1
19	140.068	32.2277	32.2277	1	1
20	120.947	24.5294	24.5294	1	1
21	97.9992	18.8201	18.8201	1	1
22	76.2888	12.5611	12.5611	1	1
23	69.9447	9.993	9.993	1	1
24	63.3859	6.93937	6.93937	1	1
25	58.4764	4.27895	4.27895	1	1
26	54.7523	3.51124	3.51124	1	1
27	49.657	7.27314	7.27314	1	1
28	42.5485	13.4652	13.4652	1	1
29	34.926	24.5802	24.5802	1	1
30	31.1416	41.0552	41.0552	1	1
31	23.0081	38.6922	38.6922	1	1
32	19.8322	44.9101	44.9101	0.0625	1

In [15]:
print(len(relaxPath))
totalPath = path+relaxPath

32


In [16]:
np.savez('pathHighRes1.npz', *totalPath)

In [27]:
loaded = np.load('path.npz')
arrays_loaded = [loaded[key] for key in loaded.files]

277
